# Init

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

# Read bronze table

In [0]:
df = spark.table("workspace.bronze.erp_erp_px_cat_g1v2")

# Transformations

## trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## Normalize maintenance to boolean flag

In [0]:
df = df.withColumn(
    "maintenance",
    F.when(F.upper(col("MAINTENANCE")) == "YES", F.lit(True))
    .when(F.upper(col("MAINTENANCE")) == "NO", F.lit(False))
    .otherwise(None)
)

## renaming columns

In [0]:
RENAME_MAP = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}

for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

# sanity check before writing to silver delta table

In [0]:
df.display()

#writing to silver table

In [0]:
df = df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_product_category")

# Sanity check for silver table

In [0]:
%sql
SELECT * FROM workspace.silver.erp_product_category LIMIT 10